In [ ]:
import rasterio
import geopandas as gpd
from shapely.geometry import Polygon
from rasterio.mask import mask
from rasterio.features import shapes
import numpy as np

In [ ]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np

def calculate_emissions(project_boundary_path, emissions_factor_path, risk_map_path):
    
    project_boundary = gpd.read_file(project_boundary_path) ## read project boundary shapefile

    with rasterio.open(emissions_factor_path) as emissions:  ## open emissions factor tif & clip it to project boundary 
        emissions_clipped, emissions_transform = rasterio.mask.mask(emissions, project_boundary.geometry, crop=True) 

    with rasterio.open(risk_map_path) as risk: ## open risk map & clip it to project boundary
        risk_clipped, risk_transform = rasterio.mask.mask(risk, project_boundary.geometry, crop=True)

    estimated_emissions = emissions_clipped * risk_clipped # calculate estimated emissions/pixel via instruction in doc section 5.6.2
    total_emissions = np.nansum(estimated_emissions)  ## sum all estimated emissions using nansum to ignore NaNs

    return total_emissions

